In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
# Функция для загрузки данных о курсе доллара за заданный диапазон дат
def get_usd_to_rub_exchange_rate(start_date, end_date):
    base_url = "https://www.cbr.ru/scripts/XML_dynamic.asp"

    # Форматирование дат для запроса
    start_date_str = start_date.strftime('%d/%m/%Y')
    end_date_str = end_date.strftime('%d/%m/%Y')

    # Параметры запроса
    params = {
        'date_req1': start_date_str,
        'date_req2': end_date_str,
        'VAL_NM_RQ': 'R01235'  # Код валюты для USD
    }

    # Запрос к API ЦБ РФ
    response = requests.get(base_url, params=params)

    if response.status_code != 200:
        raise Exception(f"Ошибка при запросе данных: {response.status_code}")

    # Парсинг ответа XML
    data = pd.read_xml(response.content, xpath="//Record")

    # Преобразование данных в DataFrame
    data['Date'] = pd.to_datetime(data['Date'], format='%d.%m.%Y')
    data['Value'] = data['Value'].str.replace(',', '.').astype(float)

    # Переименование столбцов для удобства
    data = data.rename(columns={
        'Date': 'Дата',
        'Value': 'Курс USD к RUB'
    })

    return data

# Получение данных за последние 10 лет
end_date = datetime.now()
start_date = end_date - timedelta(days=10*365)

# Загрузка данных
try:
    exchange_rate_data = get_usd_to_rub_exchange_rate(start_date, end_date)
    print("Данные успешно загружены!")
    
    # Сохранение данных в CSV
    output_file = "../../../data/external/macro/usd_to_rub.csv"
    exchange_rate_data.to_csv(output_file, index=False)
    print("Данные сохранены в файл 'usd_to_rub.csv'")
except Exception as e:
    print(f"Произошла ошибка: {e}")

In [ ]:
exchange_rate_data.head()

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import json

def get_brent_prices(api_key):
    # База API EIA
    base_url = 'https://api.eia.gov/v2/petroleum/pri/spt/data/'
    
    # Параметры запроса
    params = {
        'api_key': api_key,
        'frequency': 'weekly',  # или 'daily', 'monthly'
        'data[0]': 'value',    # значение цены
        'facets[series][]': 'RBRTE',  # код для Brent
        'sort[0][column]': 'period',
        'sort[0][direction]': 'desc',
        'length': 520  # примерно 10 лет при weekly данных
    }
    
    try:
        # Выполняем запрос
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Проверяем на ошибки
        
        # Преобразуем ответ в DataFrame
        data = response.json()
        df = pd.DataFrame(data['response']['data'])
        
        # Обрабатываем данные
        df['period'] = pd.to_datetime(df['period'])
        df = df.rename(columns={
            'period': 'Date',
            'value': 'Price'
        })
        
        # Сортируем по дате
        df = df.sort_values('Date')
        
        # Сохраняем в CSV
        df.to_csv('extra_features/brent_prices_eia.csv', index=False)
        
        print(f"Данные успешно загружены и сохранены в 'brent_prices_eia.csv'")
        print(f"Количество записей: {len(df)}")
        print("\nПервые 5 записей:")
        print(df.head())
        
        return df
        
    except requests.exceptions.RequestException as e:
        print(f"Ошибка при запросе к API: {e}")
        return None

def main():
    # Здесь нужно указать ваш API ключ от EIA
    api_key = 'CuBzqdu5DkQ0TYmFm5YumCkNgStAWZW8HcCha1De'
    
    if api_key == 'YOUR_API_KEY':
        print("Пожалуйста, укажите ваш API ключ от EIA")
        return
        
    df = get_brent_prices(api_key)

if __name__ == "__main__":
    main()

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

def download_brent_prices():
    # Получаем текущую дату
    end_date = datetime.now()
    # Дата 10 лет назад
    start_date = end_date - timedelta(days=365*10)
    
    # Загружаем данные для нефти Brent (тикер BZ=F)
    brent = yf.download('BZ=F', 
                        start=start_date.strftime('%Y-%m-%d'),
                        end=end_date.strftime('%Y-%m-%d'))
    
    # Сохраняем в CSV файл
    brent.to_csv('extra_features/brent_prices_10y.csv')
    
    print(f"Данные сохранены в файл 'brent_prices_10y.csv'")
    print(f"Количество записей: {len(brent)}")
    
    # Показываем первые несколько строк
    print("\nПервые 5 записей:")
    print(brent.head())

if __name__ == "__main__":
    download_brent_prices()